In [114]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import pickle
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from keras.layers import MaxPooling2D
from keras.applications.inception_v3 import InceptionV3

In [115]:
img_width, img_height = 100, 100

base_model = InceptionV3(weights= "imagenet",include_top = False,input_shape = (100,100,3))

In [122]:
# flat1 = Flatten()(model.outputs)
# class1 = Dense(1024, activation='relu')(flat1)
# output = Dense(10, activation='softmax')(class1)
# # define new model
# model = Model(inputs=model.inputs, outputs=output)
# # summarize
base_model.summary()

model = keras.models.Sequential()

# # Add the vgg convolutional base model
model.add(base_model)
 
# # Add new layers
# model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
 
# # Show a summary of the model. Check the number of trainable parameters    
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 49, 49, 32)   864         input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 49, 49, 32)   96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, 49, 49, 32)   0           batch_normalization_283[0][0]    
__________________________________________________________________________________________________
conv2d_284

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1, 1, 2048)        21802784  
_________________________________________________________________
dense_44 (Dense)             (None, 1, 1, 1024)        2098176   
_________________________________________________________________
dense_45 (Dense)             (None, 1, 1, 256)         262400    
_________________________________________________________________
flatten_16 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 10)                2570      
Total params: 24,165,930
Trainable params: 24,131,498
Non-trainable params: 34,432
_________________________________________________________________
None


In [123]:
train_path = "D:/GISA/Newdataset/"

In [133]:
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                   rescale=1./255,
                                   validation_split = 0.2,
                                   )

'''test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                   rescale = 1./255,)'''


train_generator = train_data_generator.flow_from_directory(
        train_path,
        target_size=(100,100),
        batch_size=32,
        class_mode='categorical',
        color_mode = 'rgb',
        shuffle = True,
        subset = 'training')

validation_generator = train_data_generator.flow_from_directory(
        train_path,
        target_size=(100,100),
        batch_size = 32,
        class_mode='categorical',
        color_mode = 'rgb',
        shuffle = True,
        subset = 'validation')
    
#'''creating check-point path to store weights of increased validation accuracy'''
        
#filepath="D:/check_points_for_keras/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

# STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
# STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
# output = model.fit_generator(train_generator ,
#                          steps_per_epoch= STEP_SIZE_TRAIN,
#                          epochs= 20,
#                          validation_data = validation_generator ,
#                          validation_steps = STEP_SIZE_VALID,
#                          verbose = 1,
#                          callbacks = callbacks_list,
#                          )

Found 9901 images belonging to 10 classes.
Found 2471 images belonging to 10 classes.


In [134]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model.fit(X, y, batch_size=32, epochs=10, validation_split=0.25)


# model.save('transfer-vgg16.model')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1, 1, 2048)        21802784  
_________________________________________________________________
dense_44 (Dense)             (None, 1, 1, 1024)        2098176   
_________________________________________________________________
dense_45 (Dense)             (None, 1, 1, 256)         262400    
_________________________________________________________________
flatten_16 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 10)                2570      
Total params: 24,165,930
Trainable params: 24,131,498
Non-trainable params: 34,432
_________________________________________________________________


In [135]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    validation_data = validation_generator, 
    validation_steps = 8,
    epochs = 20)

Epoch 1/20


ValueError: Error when checking target: expected dense_46 to have shape (1,) but got array with shape (10,)